In [3]:
import numpy as np
import torch 
from UTILS.tensor_ops import __hash__, my_view, np_one_hot, np_repeat_at, np_softmax


In [ ]:
def item_random_mv(src,dst,prob,rand=False):
    assert len(src.shape)==1; assert len(dst.shape)==1
    if rand: np.random.shuffle(src)
    len_src = len(src)
    n_mv = (np.random.rand(len_src) < prob).sum()
    print('n_mv',n_mv)
    item_mv = src[range(len_src-n_mv,len_src)]
    print('item_mv',item_mv)
    src = src[range(0,0+len_src-n_mv)]
    print('src',src)
    dst = np.concatenate((item_mv, dst))
    return src,dst

vis_index = np.array([1,2,3,4,5])
invis_index = np.array([6,7,8,9,10])
print(item_random_mv(src=vis_index, dst=invis_index, prob=0.1, rand=True))

In [2]:
reward = np.array([[-0.5, 1]]) #(time, agent)   (3,2)                                (3,0)
reward_logits = np.array([[  [[0,  0.1,  0,  0.12],[0,  0.1,  0.2,  0.1]],[[0, 0.1, 0, 0.12],[0.2, 0.1, 0, 0.12]]   ]]) #(time, agent, agent, 4)
print(reward.shape)
print(reward_logits.shape)

(1, 2)
(1, 2, 2, 4)


In [7]:
n_timestep = reward_logits.shape[0]
n_agent = reward_logits.shape[1]
assert not ((reward != 0) & (reward!=1) & (reward!=-0.5) & (reward!=0.5)).any()
# {-0.5, 0, 0.5, 1} -> {3<(-1), 0, 1, 2}
reward_projection = lambda r: np.where(r<0, 3, r*2).astype(np.int)
# apply projection
reward_class = reward_projection(reward) # np_one_hot(,4)
# ($time_step.$n_agent.) -> ($time_step.$n_agent_.$n_agent.)
reward_class = np_repeat_at(reward_class, insert_dim=1, n_times=n_agent)
# not zero reward
not_zero_mask = (reward_class!=0)
# reward_class_onehot
reward_class_onehot = np_one_hot(reward_class, 4)
# reward estimation
reward_logits = np_softmax(reward_logits, axis=-1)
prediction_rating = np.take_along_axis(reward_logits, axis=-1, 
    indices=np.expand_dims(reward_class, -1)).squeeze(-1)
correct_prediction = (np.argmax(reward_logits, -1) == reward_class)

<ipython-input-7-07a2cba73353>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  reward_projection = lambda r: np.where(r<0, 3, r*2).astype(np.int)


In [15]:
confidence_threshhold = 0.25*1.05
eye_mat = np_repeat_at(np.eye(n_agent), insert_dim=0, n_times=n_timestep)
compasion_matrix = correct_prediction & (prediction_rating > confidence_threshhold) & not_zero_mask | (eye_mat==1)
print(compasion_matrix)

[[[ True  True]
  [ True  True]]]


In [17]:
# compasion_matrix = (prediction_rating > confidence_threshhold) & not_zero_mask & (eye_mat==0)
compasion_matrix = compasion_matrix.astype(np.int)

# 计算分解后的reward，因为reward可能被多个智能体分解
n_com = compasion_matrix.sum(-2)
assert not (n_com==0).any()
reward_decmp = np.where(n_com!=0, reward/n_com, reward) # reward/(n_com+1e-7)
# https://numpy.org/doc/stable/reference/generated/numpy.matmul.html
compassion_reward = np.matmul(compasion_matrix, np.expand_dims(reward_decmp,-1)).squeeze(-1)
print(compassion_reward)

[[0.25 0.25]]


<ipython-input-17-3b9ad3e8d18f>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  compasion_matrix = compasion_matrix.astype(np.int)


In [40]:
'''
函数说明：在有限的、不均衡的多标签数据集中，按照预设的比例，取出尽可能多的样本
'''
def sample_balance(x, y, n_class, weight=None):
    if weight is None: weight = torch.ones(n_class, device=x.device)
    else: weight = torch.Tensor(weight, device=x.device)
    n_instance = torch.zeros(n_class, device=x.device)
    indices = [None]*n_class
    for i in range(n_class):
        indices[i] = torch.where(y==i)[0]
        n_instance[i] = len(indices[i])
    ratio = n_instance/weight
    bottle_neck = torch.argmin(n_instance/weight)
    r = ratio[bottle_neck]
    n_sample = (r*weight).long()
    # print(n_instance, n_sample)
    new_indices = [indices[i][torch.randperm(n_sample[i])] for i in range(n_class)]
    # print(new_indices)
    new_indices_ = torch.cat(new_indices)
    assert len(new_indices_) == sum(n_sample)
    return x[new_indices_], y[new_indices_]

'''
测试代码
'''
x = torch.rand(200, 4)
y1 = torch.rand(100, 4)
y2 = torch.rand(100, 4)
y2[:, 0] += 1
y = torch.cat((y1,y2))
y = torch.argmax(y, -1)
print(y)
n_class = 4
weight = [2,1,1,1]
sample_balance(x,y,4,weight)

tensor([0, 2, 1, 3, 3, 1, 0, 0, 1, 3, 1, 3, 2, 2, 2, 2, 3, 2, 2, 1, 0, 3, 3, 3,
        3, 2, 2, 2, 1, 3, 3, 1, 0, 0, 2, 0, 1, 2, 0, 3, 1, 3, 1, 3, 2, 3, 1, 2,
        0, 2, 1, 1, 3, 2, 3, 2, 1, 2, 1, 1, 3, 1, 0, 3, 3, 2, 0, 3, 2, 0, 2, 0,
        0, 0, 3, 2, 3, 1, 1, 1, 0, 3, 1, 3, 1, 1, 3, 1, 0, 0, 1, 2, 0, 0, 2, 1,
        0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([122.,  26.,  25.,  27.]) tensor([50, 25, 25, 25])


(tensor([[2.2656e-02, 4.1251e-01, 3.2707e-01, 7.2534e-01],
         [1.2332e-01, 2.7637e-01, 4.8348e-01, 2.7835e-01],
         [1.2237e-01, 7.0544e-01, 6.5689e-01, 5.1355e-01],
         [3.9261e-02, 6.1284e-01, 5.4430e-01, 6.9022e-02],
         [7.5210e-01, 5.6684e-01, 2.5894e-01, 5.2485e-01],
         [8.1709e-01, 8.4347e-01, 2.3117e-01, 1.1277e-01],
         [8.4153e-01, 5.2814e-01, 6.3028e-01, 2.2639e-01],
         [8.4371e-01, 6.2015e-01, 4.6847e-01, 3.5423e-02],
         [6.9924e-01, 6.0353e-01, 9.3364e-01, 1.7731e-02],
         [7.1736e-01, 9.8774e-01, 4.0972e-01, 9.4791e-01],
         [1.3912e-01, 5.7476e-02, 2.9873e-01, 6.4819e-01],
         [3.0322e-01, 5.7693e-01, 3.1014e-01, 1.8827e-01],
         [9.3742e-02, 3.8578e-01, 7.3496e-01, 1.1003e-01],
         [1.2636e-04, 2.7071e-01, 4.9675e-01, 3.1337e-01],
         [1.2764e-01, 4.1302e-02, 3.8436e-01, 2.0502e-01],
         [2.2064e-01, 2.3158e-01, 3.3987e-01, 3.1554e-01],
         [9.2388e-01, 5.5447e-01, 6.8357e-02, 5.1233e-01

In [41]:
x = torch.rand(100, 4)
y = torch.rand(100, 4)
y = torch.argmax(y, -1)

# return x[new_indices_], y[new_indices_]

tensor([21., 28., 24., 27.]) tensor([ 7, 28, 21, 14])


(tensor([[8.8547e-03, 6.8349e-01, 1.8454e-01, 5.9639e-01],
         [8.9973e-01, 7.6351e-01, 7.7360e-01, 8.7484e-02],
         [7.0845e-01, 4.7663e-01, 7.1757e-01, 1.9168e-02],
         [1.1895e-01, 1.1693e-01, 5.9661e-01, 5.2639e-01],
         [9.9516e-01, 5.8231e-01, 9.0453e-01, 3.6128e-01],
         [2.5497e-01, 2.5144e-01, 4.4595e-01, 2.2235e-01],
         [6.0990e-01, 2.6511e-01, 5.4970e-01, 2.0068e-01],
         [4.5902e-01, 9.4615e-01, 9.5732e-01, 8.1278e-01],
         [4.0710e-01, 6.8418e-02, 4.1449e-01, 5.1667e-01],
         [4.8025e-02, 6.0615e-01, 1.8411e-01, 9.6834e-01],
         [9.1179e-01, 3.8608e-01, 5.2052e-01, 5.5045e-02],
         [3.2310e-01, 9.7595e-01, 1.5475e-01, 3.1984e-01],
         [9.2913e-01, 9.3347e-01, 3.7687e-01, 8.3900e-01],
         [2.7752e-01, 3.2392e-02, 8.7160e-01, 3.2852e-01],
         [4.2996e-01, 6.1758e-01, 5.0515e-03, 3.5394e-01],
         [9.4292e-02, 9.4540e-01, 6.7231e-01, 7.4819e-01],
         [6.0542e-01, 6.2166e-01, 4.4464e-01, 8.6807e-01

NameError: name 'new_indices_' is not defined

In [2]:
import numpy as np
def my_view(x, shape):
    if -1 in shape[1:-1]: return my_view_test(x, shape)
    reverse_lookup = True if shape[0] == -1 else False
    if not reverse_lookup:
        for i, dim in enumerate(shape):
            if dim == 0:
                shape[i] = x.shape[i]
    else:
        for i in range(len(shape)):
            ni = -(i + 1)  # iter -1,-2,-3,...
            dim = shape[ni]
            if dim == 0:
                shape[ni] = x.shape[ni]
    if isinstance(x, np.ndarray):
        return x.reshape(*shape)
    return x.view(*shape)

def my_view_test(x, shape):
    # fill both way until meet -1 
    for i, dim in enumerate(shape):
        if dim == 0: shape[i] = x.shape[i]
        elif dim == -1: break
    for i in range(len(shape)):
        ni = -(i + 1); dim = shape[ni]
        if dim == 0: shape[ni] = x.shape[ni]
        elif dim == -1: break
    # print(shape)
    if isinstance(x, np.ndarray):
        return x.reshape(*shape)
    return x.view(*shape)



In [6]:

"""
    improve np.reshape and torch.view function
    If a dim is assigned with 0, it will keep its original dimension
    eg.1    x.shape = (4, 5, 6, 7); new_shape = [0, 0, -1]
            y = my_view(x, new_shape)
            y.shape = (4, 5, 6*7)

    eg.2    x.shape = (4, 5, 6, 7); new_shape = [-1, 0, 0]
            y = my_view(x, new_shape)
            y.shape = (4*5, 6, 7)

(4, 5, 6); new_shape = [0, 0, -1, 3]

(4, 5, 6); new_shape = [2, -1, 0, 0]
(3, 4, 5, 6); new_shape = [0, 2, -1, 0, 0]
"""

my_view(np.zeros(shape=(4, 5, 6)), shape = [2, -1, 0, 0]).shape

(2, 2, 5, 6)

In [8]:
my_view(np.zeros(shape=(4, 5, 6)), shape = [0, 0, -1, 3]).shape

(4, 5, 2, 3)

In [10]:
my_view(np.zeros(shape=(4, 5, 6, 7)), shape =[0, 0, -1]).shape

(4, 5, 42)